# Introduction

Our ultimate aim is to predict solar electricity power generation over the next few hours.

## Loading terabytes of data efficiently from cloud storage

We have several TB of satellite data. To keep the GPU fed with data during training, we need to read chunks of data quickly from the Zarr store; and we also want to load data asynchronously.  That is, while the GPU is training on the current batch, the data loader should simultaneously load the _next_ batch from disk.

PyTorch makes this easy!  PyTorch's `DataLoader` spawns multiple worker processes when constructed with `num_workers` set to more than 1.  Each worker process receives a copy of the `SatelliteDataset` object.

There is a small challenge: The code hangs when it gets to `enumerate(dataloader)` if we open the `xarray.DataArray` in the main process and copy that opened `DataArray` to the child processes.  Our solution is to delay the creation of the `DataArray` until _after_ the worker processes have been created.  PyTorch makes this easy by allowing us to pass a `worker_init_fn` to `DataLoader`.  `worker_init_fn` is called on each worker process.  Our `worker_init_fn` just has one job: to call `SatelliteDataset.per_worker_init()` which, in turn, opens the `DataArray`.

This approach achieves read speeds of 600 MB/s from Google Cloud Storage to a single GCP VM with 12 vCPUs (as measured by `nethogs`).

We use `IterableDataset` instead of `Dataset` so `SatelliteDataset` can pre-load the next example from disk and then block (on the `yield`) waiting for PyTorch to read that data.  This allows the worker processes to be processing the next data samples while the main process is training the current batch on the GPU.

We can't pin the memory in each worker process because pinned memory can't be shared across processes.  Instead we ask `DataLoader` to pin the collated batch so that pytorch-lightning can asynchronously load the next batch from pinned CPU memory into GPU memory.

The satellite data is stored on disk as `int16`.  To speed up the movement of the satellite data between processes and from the CPU memory to the GPU memory, we keep the data as `int16` until the data gets to the GPU, where it is converted to `float32` and normalised.

### Loading data from disk into memory in chunks

Cloud storage buckets can't seek into files like 'proper' POSIX filesystems can.  So, even if we just want 1 byte from a 1 GB file, we have to load the entire 1 GB file from the bucket.

Zarr is designed with this challenge in mind.  Zarr gets round the inability of cloud storage buckets to seek by chunking the data into lots of small files.  But, still, we have to load entire Zarr chunks at once, even if we only want a part of a chunk.  And, even though we can pull 600 MB/s from a cloud storage bucket, the reads from the storage bucket are still the rate-limiting-step.  (GPUs are very fast and have a voracious appetite for data!)

To get the most out of each disk read, our worker processes load several contiguous chunks of Zarr data from disk into memory at once.  We then randomly sample from the in-memory data multiple times, before loading another set of chunks from disk into memory.   This trick increases training speed by about 10x.

Each Zarr chunk is 36 timesteps long and contains the entire geographical extent.  Each timestep is about 5 minutes apart, so each Zarr chunk spans 1.5 hours, assuming the timesteps are contiguous (more on contiguous chunks later).

### Loading only daylight data

We're interested in forecasting solar power generation, so we don't care about nighttime data :)

In the UK in summer, the sun rises first in the north east, and sets last in the north west (see [video of June 2019](https://www.youtube.com/watch?v=IOp-tj-IJpk&t=0s)).  In summer, the north gets more hours of sunshine per day.

In the UK in winter, the sun rises first in the south east, and sets last in the south west (see [video of Jan 2019](https://www.youtube.com/watch?v=CJ4prUVa2nQ)).  In winter, the south gets more hours of sunshine per day.

|                        | Summer | Winter |
|           ---:         |  :---: |  :---: |
| Sun rises first in     | N.E.   | S.E.   |
| Sun sets last in       | N.W.   | S.W.   |
| Most hours of sunlight | North  | South  |

We always load a pre-defined number of Zarr chunks from disk every disk load (defined by `n_chunks_per_disk_load`).

Before training, we select timesteps which have at least some sunlight.  We do this by computing the clearsky global horizontal irradiance (GHI) for the four corners of the satellite imagery, and for all the timesteps in the dataset.  We only use timesteps where the maximum global horizontal irradiance across all four corners is above some threshold.

(The 'clearsky [solar irradiance](https://en.wikipedia.org/wiki/Solar_irradiance)' is the amount of sunlight we'd expect on a clear day at a specific time and location. The SI unit of irradiance is watt per square meter.  The 'global horizontal irradiance' is the total sunlight that would hit a horizontal surface on the surface of the Earth.  The GHI is the sum of the direct irradiance (sunlight which takes a direct path from the Sun to the Earth's surface) and the diffuse horizontal irradiance (the sunlight scattered from the atmosphere)).

### Finding contiguous sequences

Once we have a list of 'lit' timesteps, we then find contiguous sequences (timeseries without any gaps).  And we then compute a list of contiguous Zarr chunks that we'll load at once during training.

### Loading data during training

During training, each worker process randomly picks multiple contiguous Zarr chunk sequences from the list of contiguous sequences pre-computed before training started.  The worker loads that data into memory and then randomly samples many samples from that in-memory data before loading more data from disk.

#### Ensuring each batch contains a random sample of the dataset

When PyTorch's `DataLoader` constructs a batch, it reads from just one worker process.  (This is not how I had _assumed_ it would work:  I assumed PyTorch would construct each batch by randomly sampling from all workers.)  This is an issue because, for stochastic gradient descent to work correctly, each batch must contain random samples of the dataset.  So it's not sufficient for each worker to load just one contiguous Zarr chunk (because then each batch would be made up entirely of samples from roughly the same time of day).  So, instead, each worker process loads multiple contiguous Zarr sequences into memory.  This also means that each worker must load quite a lot of data from disk.  To avoid training pausing while a worker process loads more data from disk, the data loading is done asynchronously using a separate thread within each worker process.

## Timestep numbering:

* t<sub>0</sub> is 'now': the most recent observation.
* t<sub>1</sub> is the first timestep of the forecast.

In [1]:
# Python core
from typing import Optional, Callable, TypedDict, Union, Iterable, Tuple, NamedTuple, List
from dataclasses import dataclass
import datetime
from itertools import product
from concurrent import futures
import multiprocessing  # Just for setting start_method

# Necessary, otherwise the code hangs when we try to use gcsfs from multiple processes!
# See https://github.com/dask/gcsfs/issues/379
# Must be called before PyTorch in imported.
multiprocessing.set_start_method('spawn')

# Scientific python
import numpy as np
import pandas as pd
import xarray as xr
import numcodecs
import matplotlib.pyplot as plt

# Cloud compute
import gcsfs

# PyTorch
import torch
from torch import nn
import torch.nn.functional as F
import pytorch_lightning as pl

# PV & geospatial
import pvlib
import pyproj

In [2]:
xr.__version__

'0.17.0'

In [3]:
gcsfs.__version__

'2021.04.0'

In [4]:
torch.__version__

'1.8.1'

In [5]:
torch.cuda.is_available()

True

In [6]:
pyproj.__version__

'3.0.1'

## Consts & config

The [Zarr docs](https://zarr.readthedocs.io/en/stable/tutorial.html#configuring-blosc) say we should tell the Blosc compression library not to use threads because we're using multiple processes to read from our Zarr store:

In [7]:
numcodecs.blosc.use_threads = False

ZARR = 'solar-pv-nowcasting-data/satellite/EUMETSAT/SEVIRI_RSS/OSGB36/all_zarr_int16'

plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.interpolation'] = 'none'

torch.cuda.is_available()

True

# Load satellite data

In [8]:
def get_sat_data(filename: str=ZARR) -> xr.DataArray:
    """Lazily opens the Zarr store on Google Cloud Storage (GCS).
    
    Selects the High Resolution Visible (HRV) satellite channel.
    """
    print('get_sat_data(filename=', filename, ')')
    gcs = gcsfs.GCSFileSystem() # access='read_only')
    print('gcs')
    store = gcsfs.GCSMap(root=filename, gcs=gcs)
    print('store')
    dataset = xr.open_zarr(store, consolidated=True)
    print('dataset')
    return dataset['stacked_eumetsat_data'].sel(variable='HRV')

In [9]:
#sat_data.close(); del sat_data

In [10]:
%%time
sat_data = get_sat_data()

get_sat_data(filename= solar-pv-nowcasting-data/satellite/EUMETSAT/SEVIRI_RSS/OSGB36/all_zarr_int16 )
gcs
store
dataset
CPU times: user 1.69 s, sys: 81.3 ms, total: 1.77 s
Wall time: 3.82 s


Caution: Wierdly, plotting `sat_data` at this point causes the code to hang (with no errors messages) when it gets to `for batch in dataloader:`.  The code hangs even if we first do `sat_data.close(); del sat_data`

In [11]:
# sat_data

Get the timestep indicies of each Zarr chunk.  Later, we will use these boundaries to ensure we load complete chunks at a time.

In [12]:
zarr_chunk_boundaries = np.concatenate(([0], np.cumsum(sat_data.chunks[0])))

## Select daylight hours

In [13]:
# OSGB is also called "OSGB 1936 / British National Grid -- United Kingdom Ordnance Survey".
# OSGB is used in many UK electricity system maps, and is used by the UK Met Office UKV model.
# OSGB is a Transverse Mercator projection, using 'easting' and 'northing' coordinates
# which are in meters.
OSGB = 27700

# WGS84 is short for "World Geodetic System 1984", used in GPS. Uses latitude and longitude.
WGS84 = 4326

# osgb_to_wgs84.transform() returns latitude (north-south), longitude (east-west)
osgb_to_wgs84 = pyproj.Transformer.from_crs(crs_from=OSGB, crs_to=WGS84)


def get_daylight_timestamps(
    dt_index: pd.DatetimeIndex, 
    locations: Iterable[Tuple[float, float]],
    ghi_threshold: float = 1
    ) -> pd.DatetimeIndex:
    """Returns datetimes for which the global horizontal irradiance
    (GHI) is above ghi_threshold across all locations.
    
    Args:
      dt_index: DatetimeIndex to filter.  Must be UTC.
      locations: List of Tuples of x, y coordinates in OSGB projection.
      ghi_threshold: Global horizontal irradiance threshold.
    """
    assert dt_index.tz.zone == 'UTC'
    ghi_for_all_locations = []
    for x, y in locations:
        lat, lon = osgb_to_wgs84.transform(x, y)
        location = pvlib.location.Location(latitude=lat, longitude=lon)
        clearsky = location.get_clearsky(dt_index)
        ghi = clearsky['ghi']
        ghi_for_all_locations.append(ghi)
        
    ghi_for_all_locations = pd.concat(ghi_for_all_locations, axis='columns')
    max_ghi = ghi_for_all_locations.max(axis='columns')
    mask = max_ghi > ghi_threshold
    return dt_index[mask]

In [14]:
GEO_BORDER: int = 64  #: In same geo projection and units as sat_data.
corners = [
    (sat_data.x.values[x], sat_data.y.values[y]) 
    for x, y in product(
        [GEO_BORDER, -GEO_BORDER], 
        [GEO_BORDER, -GEO_BORDER])]

In [15]:
%%time
datetimes = get_daylight_timestamps(
    dt_index=pd.DatetimeIndex(sat_data.time.values, tz='UTC'), 
    locations=corners)

CPU times: user 6.67 s, sys: 95.1 ms, total: 6.76 s
Wall time: 6.62 s


In [16]:
#sat_data.close()
#del sat_data

## Get contiguous segments of satellite data

In [17]:
class Segment(NamedTuple):
    """Represents the start and end indicies of a segment of contiguous samples."""
    start: int
    end: int


def get_contiguous_segments(dt_index: pd.DatetimeIndex, min_timesteps: int, max_gap: pd.Timedelta) -> Iterable[Segment]:
    """Chunk datetime index into contiguous segments, each at least min_timesteps long.
    
    max_gap defines the threshold for what constitutes a 'gap' between contiguous segments.
    
    Throw away any timesteps in a sequence shorter than min_timesteps long.
    """
    gap_mask = np.diff(dt_index) > max_gap
    gap_indices = np.argwhere(gap_mask)[:, 0]

    # gap_indicies are the indices into dt_index for the timestep immediately before the gap.
    # e.g. if the datetimes at 12:00, 12:05, 18:00, 18:05 then gap_indicies will be [1].
    segment_boundaries = gap_indices + 1

    # Capture the last segment of dt_index.
    segment_boundaries = np.concatenate((segment_boundaries, [len(dt_index)]))

    segments = []
    start_i = 0
    for end_i in segment_boundaries:
        n_timesteps = end_i - start_i
        if n_timesteps >= min_timesteps:
            segment = Segment(start=start_i, end=end_i)
            segments.append(segment)
        start_i = end_i
        
    return segments

In [18]:
%%time
contiguous_segments = get_contiguous_segments(
    dt_index = datetimes,
    min_timesteps = 36 * 1.5,
    max_gap = pd.Timedelta('5 minutes'))

contiguous_segments[:5]

CPU times: user 781 ms, sys: 6.57 ms, total: 788 ms
Wall time: 786 ms


[Segment(start=0, end=222),
 Segment(start=222, end=445),
 Segment(start=445, end=669),
 Segment(start=669, end=893),
 Segment(start=893, end=1117)]

In [19]:
len(contiguous_segments)

598

## Turn the contiguous segments into sequences of Zarr chunks, which will be loaded together during training

In [20]:
def get_zarr_chunk_sequences(
    n_chunks_per_disk_load: int, 
    zarr_chunk_boundaries: Iterable[int], 
    contiguous_segments: Iterable[Segment]) -> Iterable[Segment]:
    """
    
    Args:
      n_chunks_per_disk_load: Maximum number of Zarr chunks to load from disk in one go.
      zarr_chunk_boundaries: The indicies into the Zarr store's time dimension which define the Zarr chunk boundaries.
        Must be sorted.
      contiguous_segments: Indicies into the Zarr store's time dimension that define contiguous timeseries.
        That is, timeseries with no gaps.
    
    Returns zarr_chunk_sequences: a list of Segments representing the start and end indicies of contiguous sequences of multiple Zarr chunks,
    all exactly n_chunks_per_disk_load long (for contiguous segments at least as long as n_chunks_per_disk_load zarr chunks),
    and at least one side of the boundary will lie on a 'natural' Zarr chunk boundary.
    
    For example, say that n_chunks_per_disk_load = 3, and the Zarr chunks sizes are all 5:
    
    
                  0    5   10   15   20   25   30   35 
                  |....|....|....|....|....|....|....|

    INPUTS:
                     |------CONTIGUOUS SEGMENT----|
                     
    zarr_chunk_boundaries:
                  |----|----|----|----|----|----|----|
    
    OUTPUT:
    zarr_chunk_sequences:
           3 to 15:  |-|----|----|
           5 to 20:    |----|----|----|
          10 to 25:         |----|----|----|
          15 to 30:              |----|----|----|
          20 to 32:                   |----|----|-|
    
    """
    assert n_chunks_per_disk_load > 0
    
    zarr_chunk_sequences = []

    for contig_segment in contiguous_segments:
        # searchsorted() returns the index into zarr_chunk_boundaries at which contig_segment.start
        # should be inserted into zarr_chunk_boundaries to maintain a sorted list.
        # i_of_first_zarr_chunk is the index to the element in zarr_chunk_boundaries which defines
        # the start of the current contig chunk.
        i_of_first_zarr_chunk = np.searchsorted(zarr_chunk_boundaries, contig_segment.start)
        
        # i_of_first_zarr_chunk will be too large by 1 unless contig_segment.start lies
        # exactly on a Zarr chunk boundary.  Hence we must subtract 1, or else we'll
        # end up with the first contig_chunk being 1 + n_chunks_per_disk_load chunks long.
        if zarr_chunk_boundaries[i_of_first_zarr_chunk] > contig_segment.start:
            i_of_first_zarr_chunk -= 1
            
        # Prepare for looping to create multiple Zarr chunk sequences for the current contig_segment.
        zarr_chunk_seq_start_i = contig_segment.start
        zarr_chunk_seq_end_i = None  # Just a convenience to allow us to break the while loop by checking if zarr_chunk_seq_end_i != contig_segment.end.
        while zarr_chunk_seq_end_i != contig_segment.end:
            zarr_chunk_seq_end_i = zarr_chunk_boundaries[i_of_first_zarr_chunk + n_chunks_per_disk_load]
            zarr_chunk_seq_end_i = min(zarr_chunk_seq_end_i, contig_segment.end)
            zarr_chunk_sequences.append(Segment(start=zarr_chunk_seq_start_i, end=zarr_chunk_seq_end_i))
            i_of_first_zarr_chunk += 1
            zarr_chunk_seq_start_i = zarr_chunk_boundaries[i_of_first_zarr_chunk]
            
    return zarr_chunk_sequences

In [21]:
zarr_chunk_sequences = get_zarr_chunk_sequences(
    n_chunks_per_disk_load=3,
    zarr_chunk_boundaries=zarr_chunk_boundaries,
    contiguous_segments=contiguous_segments)

zarr_chunk_sequences[:10]

[Segment(start=0, end=108),
 Segment(start=36, end=144),
 Segment(start=72, end=180),
 Segment(start=108, end=216),
 Segment(start=144, end=222),
 Segment(start=222, end=324),
 Segment(start=252, end=360),
 Segment(start=288, end=396),
 Segment(start=324, end=432),
 Segment(start=360, end=445)]

## PyTorch data storage & processing

In [22]:
Array = Union[np.ndarray, xr.DataArray]

IMAGE_ATTR_NAMES = ('historical_sat_images', 'target_sat_images')

class Sample(TypedDict):
    """Simple class for structuring data for the ML model.
    
    Using typing.TypedDict gives us several advantages:
      1. Single 'source of truth' for the type and documentation of each example.
      2. A static type checker can check the types are correct.

    Instead of TypedDict, we could use typing.NamedTuple,
    which would provide runtime checks, but the deal-breaker with Tuples is that they're immutable
    so we cannot change the values in the transforms.
    """
    # IMAGES
    # Shape: batch_size, seq_length, width, height
    historical_sat_images: Array
    target_sat_images: Array
        
    # METADATA
    datetime_index: Array


class BadData(Exception):
    pass


@dataclass
class RandomSquareCrop():
    size: int = 128  #: Size of the cropped image.

    def __call__(self, sample: Sample) -> Sample:
        crop_params = None
        for attr_name in IMAGE_ATTR_NAMES:
            image = sample[attr_name]
            # TODO: Random crop!
            cropped_image = image[..., :self.size, :self.size]
            sample[attr_name] = cropped_image
        return sample


class CheckForBadData():
    def __call__(self, sample: Sample) -> Sample:
        for attr_name in IMAGE_ATTR_NAMES:
            image = sample[attr_name]
            if np.any(image < 0):
                raise BadData(f'\n{attr_name} has negative values at {image.time.values}')
        return sample

        
class ToTensor():
    def __call__(self, sample: Sample) -> Sample:
        for key, value in sample.items():
            if isinstance(value, xr.DataArray):
                value = value.values
            sample[key] = torch.from_numpy(value)
        return sample
    
    
class Compose():
    # Copied from https://pytorch.org/vision/stable/_modules/torchvision/transforms/transforms.html#Compose
    # But not using torchvision, because it appears to create conda package conflicts
    # with opencv?  But need to explore more!
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, img):
        for t in self.transforms:
            img = t(img)
        return img

## PyTorch dataset

In [23]:
@dataclass
class SatelliteDataset(torch.utils.data.IterableDataset):
    zarr_chunk_sequences: Iterable[Segment]  #: Defines multiple Zarr chunks to be loaded from disk at once.
    history_len: int = 1  #: The number of timesteps of 'history' to load.
    forecast_len: int = 1  #: The number of timesteps of 'forecast' to load.
    transform: Optional[Callable] = None
    n_disk_loads_per_epoch: int = 10_000  #: Number of disk loads per worker process per epoch.
    min_n_samples_per_disk_load: int = 1_000  #: Number of samples each worker will load for each disk load.
    max_n_samples_per_disk_load: int = 2_000  #: Max number of disk loader.  Actual number is chosen randomly between min & max.
    n_zarr_chunk_sequences_to_load_at_once: int = 8  #: Number of chunk seqs to load at once.  These are sampled at random.
    
    def __post_init__(self):
        #: Total sequence length of each sample.
        self.total_seq_len = self.history_len + self.forecast_len

    def per_worker_init(self, worker_id: int) -> None:
        """Called by worker_init_fn on each copy of SatelliteDataset after the worker process has been spawned."""
        print('Starting per_worker_init for worker', worker_id)
        self.worker_id = worker_id
        print('Starting to load sat data for worker', worker_id)
        self.data_array = get_sat_data()
        print('Finished loading sat data for worker', worker_id)
        # Each worker must have a different seed for its random number generator.
        # Otherwise all the workers will output exactly the same data!
        seed = torch.initial_seed()
        self.rng = np.random.default_rng(seed=seed)
        print('Finished per_worker_init for worker', worker_id)
    
    def __iter__(self):
        """
        Asynchronously loads next data from disk while sampling from data_in_mem.
        """
        with futures.ThreadPoolExecutor(max_workers=1) as executor:
            future_data = executor.submit(self._load_data_from_disk)
            for _ in range(self.n_disk_loads_per_epoch):
                data_in_mem = future_data.result()
                future_data = executor.submit(self._load_data_from_disk)
                n_samples = self.rng.integers(self.min_n_samples_per_disk_load, self.max_n_samples_per_disk_load)
                for _ in range(n_samples):
                    sample = self._get_sample(data_in_mem)
                    if self.transform:
                        try:
                            sample = self.transform(sample)
                        except BadData as e:
                            print(e)
                            continue
                    yield sample

    def _load_data_from_disk(self) -> List[xr.DataArray]:
        """Loads data from contiguous Zarr chunks from disk into memory."""
        sat_images_list = []
        for _ in range(self.n_zarr_chunk_sequences_to_load_at_once):
            zarr_chunk_sequence = self.rng.choice(self.zarr_chunk_sequences)
            sat_images = self.data_array.isel(time=slice(*zarr_chunk_sequence))

            # Sanity checks
            n_timesteps_available = len(sat_images)
            if n_timesteps_available < self.total_seq_len:
                raise RuntimeError(f'Not enough timesteps in loaded data!  Need at least {self.total_seq_len}.  Got {n_timesteps_available}!')

            sat_images_list.append(sat_images.load())
        return sat_images_list

    def _get_sample(self, data_in_mem_list: List[xr.DataArray]) -> Sample:
        i = self.rng.integers(0, len(data_in_mem_list))
        data_in_mem = data_in_mem_list[i]
        n_timesteps_available = len(data_in_mem)
        max_start_idx = n_timesteps_available - self.total_seq_len
        start_idx = self.rng.integers(low=0, high=max_start_idx, dtype=np.uint32)
        end_idx = start_idx + self.total_seq_len
        sat_images = data_in_mem.isel(time=slice(start_idx, end_idx))
        return Sample(
            historical_sat_images=sat_images[:self.history_len],
            target_sat_images=sat_images[self.history_len:],
            datetime_index=sat_images.time.values.astype('datetime64[s]').astype(int)
        )


def worker_init_fn(worker_id):
    """Configures each dataset worker process.
    
    Just has one job!  To call SatelliteDataset.per_worker_init().
    """
    # get_worker_info() returns information specific to each worker process.
    worker_info = torch.utils.data.get_worker_info()
    if worker_info is None:
        print('worker_info is None!')
    else:
        dataset_obj = worker_info.dataset  # The Dataset copy in this worker process.
        dataset_obj.per_worker_init(worker_id=worker_info.id)
        print('worker', worker_info.id, 'initialised!')


torch.manual_seed(42)    

dataset = SatelliteDataset(
    zarr_chunk_sequences=zarr_chunk_sequences,
    transform=Compose([
        RandomSquareCrop(),
        CheckForBadData(),
        ToTensor(),
    ]),
)

In [24]:
dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=8,  # Was 8!
    num_workers=8,  # Was 8!  # timings:  4=13.8s; 8=11.6; 10=11.3s; 11=11.5s; 12=12.6s.  10=3it/s
    worker_init_fn=worker_init_fn,
    pin_memory=False,
    #persistent_workers=True
)

In [25]:
%%time
for batch in dataloader:
    print(batch['historical_sat_images'].shape)
    break

RuntimeError: DataLoader worker (pid(s) 67567) exited unexpectedly

In [26]:
pd.to_datetime(batch['datetime_index'].numpy().flatten(), unit='s').values.reshape(-1, 2).astype('datetime64[s]')

NameError: name 'batch' is not defined

In [ ]:
batch['historical_sat_images'].shape

In [ ]:
batch['target_sat_images'].shape

In [ ]:
batch['historical_sat_images'].dtype

In [ ]:
plt.imshow(batch['historical_sat_images'][4, 0])

# Simple ML model

In [ ]:
def normalise_images_in_model(images, device):    
    SAT_IMAGE_MEAN = torch.tensor(93.23458, dtype=torch.float, device=device)
    SAT_IMAGE_STD = torch.tensor(115.34247, dtype=torch.float, device=device)
    
    images = images.float()
    images -= SAT_IMAGE_MEAN
    images /= SAT_IMAGE_STD
    return images

In [ ]:
CHANNELS = 32
KERNEL = 3


class LitAutoEncoder(pl.LightningModule):
    def __init__(self):
        super().__init__()
        
        self.encoder_conv1 = nn.Conv2d(in_channels=1, out_channels=CHANNELS//2, kernel_size=KERNEL)
        self.encoder_conv2 = nn.Conv2d(in_channels=CHANNELS//2, out_channels=CHANNELS, kernel_size=KERNEL)
        self.encoder_conv3 = nn.Conv2d(in_channels=CHANNELS, out_channels=CHANNELS, kernel_size=KERNEL)
        self.encoder_conv4 = nn.Conv2d(in_channels=CHANNELS, out_channels=CHANNELS, kernel_size=KERNEL)

        self.maxpool = nn.MaxPool2d(kernel_size=KERNEL)
        
        self.decoder_conv1 = nn.ConvTranspose2d(in_channels=CHANNELS, out_channels=CHANNELS, kernel_size=KERNEL)
        self.decoder_conv2 = nn.ConvTranspose2d(in_channels=CHANNELS, out_channels=CHANNELS//2, kernel_size=KERNEL)
        self.decoder_conv3 = nn.ConvTranspose2d(in_channels=CHANNELS//2, out_channels=CHANNELS//2, kernel_size=KERNEL)
        self.decoder_conv4 = nn.ConvTranspose2d(in_channels=CHANNELS//2, out_channels=1, kernel_size=KERNEL)
        
    def forward(self, x):
        images = x['historical_sat_images']
        images = normalise_images_in_model(images, self.device)
        
        # Pass data through the network :)
        # ENCODER
        out = F.relu(self.encoder_conv1(images))        
        out = F.relu(self.encoder_conv2(out))       
        out = F.relu(self.encoder_conv3(out))
        out = F.relu(self.encoder_conv4(out))
        out = self.maxpool(out)
        
        # DECODER
        out = F.relu(self.decoder_conv1(out))        
        out = F.relu(self.decoder_conv2(out))        
        out = F.relu(self.decoder_conv3(out))
        out = self.decoder_conv4(out)
        
        return out
    
    def _training_or_validation_step(self, batch, is_train_step):
        y_hat = self(batch)
        y = batch['target_sat_images']
        y = normalise_images_in_model(y, self.device)
        y = y[..., 40:-40, 40:-40]  # Due to the CNN stride, the output image is 48 x 48
        loss = F.mse_loss(y_hat, y)
        tag = "Loss/Train" if is_train_step else "Loss/Validation"
        self.log_dict({tag: loss}, on_step=is_train_step, on_epoch=True)
        return loss

    def training_step(self, batch, batch_idx):
        return self._training_or_validation_step(batch, is_train_step=True)
    
    def validation_step(self, batch, batch_idx):
        return self._training_or_validation_step(batch, is_train_step=False)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        return optimizer

In [ ]:
model = LitAutoEncoder()

In [ ]:
trainer = pl.Trainer(gpus=1, max_epochs=400, terminate_on_nan=False)

In [ ]:
%%time
trainer.fit(model, train_dataloader=dataloader)